In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Topología Equipo 5/Archivos Reto Topología')

In [3]:
import os
os.listdir('/content/drive/My Drive/Topología Equipo 5')


['Archivos Reto Topología', 'Actividades']

In [4]:
from generate_datasets import make_gravitational_waves
import numpy as np
from google.colab import files
from pathlib import Path

DATA = Path('/content/drive/My Drive/Topología Equipo 5/Archivos Reto Topología/data')
DATA.mkdir(parents=True, exist_ok=True)

n_signals = 500

r1=0.65
intervalo = 0.1

all_noisy_signals = []
all_gw_signals = []
all_labels = []
i=0
for _ in range(5):
    R1 = r1-i
    noisy_signals, gw_signals, labels = make_gravitational_waves(
        path_to_data=DATA, n_signals=n_signals, r_min=R1, r_max=R1, n_snr_values=1
    )
    all_noisy_signals.append(noisy_signals)
    all_gw_signals.append(gw_signals)
    all_labels.append(labels)
    i=i+0.1

all_noisy_signals = np.concatenate(all_noisy_signals)
all_gw_signals = np.concatenate(all_gw_signals)
all_labels = np.concatenate(all_labels)

np.save( 'all_noisy_signals_ambos.npy', all_noisy_signals)
np.save( 'all_gw_signals_ambos.npy', all_gw_signals)
np.save( 'all_labels_ambos.npy', all_labels)

files.download('all_noisy_signals_ambos.npy')
files.download('all_gw_signals_ambos.npy')
files.download('all_labels_ambos.npy')

In [5]:
noisy_signals= all_noisy_signals
gw_signals=all_gw_signals
labels= all_labels

print("Number of noisy signals:", len(noisy_signals))
print("Number of timesteps per series:", len(noisy_signals[0]))

Number of noisy signals: 2500
Number of timesteps per series: 8692


In [6]:
pip install pycdb

  Preparing metadata (setup.py) ... done
  Created wheel for pycdb: filename=pycdb-0.0.1-cp310-cp310-linux_x86_64.whl size=181366 sha256=c8517919b4a5b76991ce06e33ccbf1ec8abb38957e5687825a4f738da751354c
  Stored in directory: /root/.cache/pip/wheels/82/f1/61/69e77e66d9e3650cd8a85395d48c74f7d1d67010fe5d6a9dd6
Successfully built pycdb


In [8]:
2import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# get the index corresponding to the first pure noise time series
background_idx = np.argmin(labels)
# get the index corresponding to the first noise + gravitational wave time series
signal_idx = np.argmax(labels)

ts_noise = noisy_signals[background_idx]
ts_background = noisy_signals[signal_idx]
ts_signal = gw_signals[signal_idx]

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_noise))), y=ts_noise, mode="lines", name="noise"),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(ts_background))),
        y=ts_background,
        mode="lines",
        name="background",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_signal))), y=ts_signal, mode="lines", name="signal"),
    row=1,
    col=2,
)
fig.show()

In [9]:
!pip install umap-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.7 MB/s eta 0:00:00


In [10]:
pip install giotto-tda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.6/554.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [11]:
from gtda.time_series import SingleTakensEmbedding
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_gw_embedded = embedder.fit_transform(gw_signals[0])

In [12]:
from umap import UMAP
from gtda.plotting import plot_point_cloud

umap = UMAP(n_components=3, n_neighbors=15, min_dist=0.001, metric='euclidean')

y_gw_embedded_umap = umap.fit_transform(y_gw_embedded)
plot_point_cloud(y_gw_embedded_umap)


In [13]:
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud

pca = PCA(n_components=3)
y_gw_embedded_pca = pca.fit_transform(y_gw_embedded)

plot_point_cloud(y_gw_embedded_pca)

In [14]:
from sklearn.manifold import TSNE
from gtda.plotting import plot_point_cloud

# Configuración de t-SNE
tsne = TSNE(n_components=3, perplexity=30, learning_rate=200)

# Transformar los datos
y_gw_embedded_tsne = tsne.fit_transform(y_gw_embedded)

# Visualización del resultado
plot_point_cloud(y_gw_embedded_tsne)


In [15]:
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])

tsne = TSNE(n_components=3, perplexity=30, learning_rate=200)

y_noise_embedded_pca = tsne.fit_transform(y_noise_embedded)

plot_point_cloud(y_noise_embedded_pca)

In [16]:
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])

y_noise_embedded_pca = umap.fit_transform(y_noise_embedded)

plot_point_cloud(y_noise_embedded_pca)

In [17]:
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])

pca = PCA(n_components=3)
y_noise_embedded_pca = pca.fit_transform(y_noise_embedded)

plot_point_cloud(y_noise_embedded_pca)